# テキストの埋め込みを使った類似性検索デモノートブック
埋め込みモデル：Cohere Embed Multilingual v 3.0(OCI Generative AI Searvice)

In [1]:
import oci
import os
from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd

In [2]:
_= load_dotenv(find_dotenv())

## OCI認証パラメータの設定

In [3]:
CONFIG_PROFILE = "DEFAULT" # 構成ファイルに合わせて変更してください。
config = oci.config.from_file(file_location='~/.oci/config', profile_name=CONFIG_PROFILE)
config["region"] = "us-chicago-1"

## OCI Generative AIクライアントの生成

In [4]:
compartment_id = os.getenv("OCI_COMPARTMENT_ID") 

In [5]:
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))

## 埋め込み処理共通のパラメータの設定

In [6]:
model_id = "cohere.embed-multilingual-v3.0" # 埋め込みモデル

In [7]:
# 埋め込み処理の詳細なパラメータを設定するオブジェクトを生成
embed_text_detail = oci.generative_ai_inference.models.EmbedTextDetails()
embed_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id=model_id)
embed_text_detail.compartment_id = compartment_id

## 検索対象ドキュメントの埋め込み処理

### 検索対象のドキュメントの定義

In [8]:
documents = [
    "COVID-19には多くの症状があります。",
    "COVID-19の症状は悪いです。",
    "COVID-19の症状は良くありません",
    "COVID-19の症状はいろいろです",
    "COVID-19の症状は悪いです。5Gの機能には、より広範なサービスカバレッジ、より多くの利用可能な接続数、および低電力消費が含まれます。",
    "COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。",
    "COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "認知症には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COVID-19には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。",
    "新型コロナ感染症の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "鳥インフルエンザの症状には、高熱、咳、筋肉の痛み、倦怠感、頭痛、その他の症状が含まれます。",
    "COVID-19は、SARS-CoV-2（新型コロナウイルス）によって引き起こされる感染症です。2019年12月に最初に確認され、その後、世界的なパンデミックとなりました。症状としては、発熱、咳、息切れ、倦怠感、嗅覚や味覚の喪失などがあります。また、感染経路は、主に飛沫感染と接触感染ですが、空気感染の可能性もあります。高齢者や基礎疾患のある方は重症化リスクが高く、死亡率も高いとされています。COVID-19に対しては、ワクチン接種や手洗い、マスク着用、ソーシャルディスタンスなどの予防措置が推奨されています。パンデミックは世界中の医療システムや経済に大きな影響を与えました。",
    "SGDs（持続可能な開発目標）は、2015年に国連加盟国によって採択された「持続可能な開発のための2030アジェンダ」の中核を成す17の国際目標です。",
    "ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。",
    "著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。",
    "ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。",
    "近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。",
    "Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです"
]

In [9]:
embed_text_detail.inputs = documents

### ドキュメントの埋め込み実行

In [10]:
embed_text_detail.input_type = "SEARCH_DOCUMENT" # 検索対象のドキュメント
doc_embedding = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
doc_embedding_vectors = np.array([embed for embed in doc_embedding.data.embeddings])

In [11]:
print(f"埋め込みの数:{len(doc_embedding.data.embeddings)}")
print(f"埋め込みの次元数:{len(doc_embedding.data.embeddings[0])}")

埋め込みの数:19
埋め込みの次元数:1024


In [12]:
print(f"1件目の埋め込み (最初の5要素):{doc_embedding.data.embeddings[0][:5]}")

1件目の埋め込み (最初の5要素):[-0.0001835823, -0.0014657974, 0.03866577, -0.0029411316, 0.006828308]


## 類似性検索のデモ（1）

### クエリーの埋め込み処理

#### クエリーの定義

In [13]:
query1 = ["ベクトルデータベースとは？"]

#### クエリー1の埋め込み生成

In [14]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector1 = np.array(query_embedding1.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [15]:
scores = np.dot(query_vector1, np.transpose(doc_embedding_vectors))

#### 類似度順に並べ替えて上位5件を抽出

In [16]:
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n] # scores の要素を降順にソートしてインデックスを返す

#### 結果の表示

In [17]:
# Pandas Dataframe を使って結果を表示
df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.8247,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
1,2,0.7664,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
2,3,0.6599,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
3,4,0.5685,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです
4,5,0.4050,COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。


## 類似性検索のデモ（2）

### クエリーの埋め込み処理

#### クエリーの定義

In [18]:
query2 = ["近似最近傍検索とは？"]

#### クエリー2の埋め込み生成

In [19]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query2
query_embedding2 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector2 = np.array(query_embedding2.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [20]:
# 類似度スコアの計算
scores = np.dot(query_vector2, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [21]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.8964,近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。
1,2,0.6872,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
2,3,0.4275,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
3,4,0.3810,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
4,5,0.3495,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです


## 類似性検索のデモ（3） - おまけ（クエリーの埋め込みの平均ベクトルで検索してみる）

### クエリーの埋め込み処理

#### クエリーの定義

In [22]:
query1 = ["ベクトルデータベースとは？"]
query2 = ["近似最近傍検索とは？"]

#### クエリー1、2の埋め込み実行

In [23]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
embed_text_detail.inputs = query2
query_embedding2 = generative_ai_inference_client.embed_text(embed_text_detail)

#### クエリー1、2の埋め込みの平均を計算

In [24]:
query_vector1 = np.array(query_embedding1.data.embeddings[0])
query_vector2 = np.array(query_embedding2.data.embeddings[0])
avg_query_vector = (query_vector1 + query_vector2) / 2
print(f"埋め込みの平均のノルム（ベクトルの長さ）: {np.linalg.norm(avg_query_vector)}")
query_embedding_vector = avg_query_vector / np.linalg.norm(avg_query_vector)
print(f"正規化した埋め込みのノルムベクトルの長さ）: {np.linalg.norm(query_embedding_vector)}")

埋め込みの平均のノルム（ベクトルの長さ）: 0.8370207499170155
正規化した埋め込みのノルムベクトルの長さ）: 1.0


### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [25]:
# レスポンスオブジェクトから埋め込みベクトルを抽出
doc_embedding_vectors = np.array([embed for embed in doc_embedding.data.embeddings])
# 類似度スコアの計算
scores = np.dot(query_embedding_vector, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [26]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.8046,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
1,2,0.7724,近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。
2,3,0.7482,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
3,4,0.6855,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
4,5,0.5484,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです


## 類似性検索のデモ（4） - クロスリンガル（言語跨りの）類似性検索

### クエリーの埋め込み処理

#### クエリーの定義

In [27]:
query1 = ["What is a vector database?"]
#query1 = ["¿Qué es una base de datos vectorial?"] #スペイン語

#### クエリー1の埋め込み生成

In [28]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector1 = np.array(query_embedding1.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [29]:
# 類似度スコアの計算
scores = np.dot(query_vector1, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [30]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.5700,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
1,2,0.5264,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
2,3,0.4734,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです
3,4,0.4087,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
4,5,0.3009,COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。


## 類似性検索のデモ（5） - 多言語混在ドキュメントの類似性検索

### 検索対象のドキュメントの定義

In [31]:
documents = [
    "COVID-19には多くの症状があります。",
    "COVID-19の症状は悪いです。",
    "COVID-19の症状は良くありません",
    "COVID-19の症状はいろいろです",
    "COVID-19の症状は悪いです。5Gの機能には、より広範なサービスカバレッジ、より多くの利用可能な接続数、および低電力消費が含まれます。",
    "COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。",
    "COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "認知症には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COVID-19には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。",
    "新型コロナ感染症の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "鳥インフルエンザの症状には、高熱、咳、筋肉の痛み、倦怠感、頭痛、その他の症状が含まれます。",
    "COVID-19は、SARS-CoV-2（新型コロナウイルス）によって引き起こされる感染症です。2019年12月に最初に確認され、その後、世界的なパンデミックとなりました。症状としては、発熱、咳、息切れ、倦怠感、嗅覚や味覚の喪失などがあります。また、感染経路は、主に飛沫感染と接触感染ですが、空気感染の可能性もあります。高齢者や基礎疾患のある方は重症化リスクが高く、死亡率も高いとされています。COVID-19に対しては、ワクチン接種や手洗い、マスク着用、ソーシャルディスタンスなどの予防措置が推奨されています。パンデミックは世界中の医療システムや経済に大きな影響を与えました。",
    "SGDs（持続可能な開発目標）は、2015年に国連加盟国によって採択された「持続可能な開発のための2030アジェンダ」の中核を成す17の国際目標です。",
    "ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。",
    "著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。",
    "ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。",
    "近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。",
    "Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです",
    "COVID-19 has many symptoms.",
    "The symptoms of COVID-19 are bad.",
    "The symptoms of COVID-19 are not good.",
    "The symptoms of COVID-19 are various.",
    "The symptoms of COVID-19 are bad. The features of 5G include broader service coverage, more available connections, and low power consumption.",
    "COVID-19 is a disease caused by a virus. The most common symptoms are fever, chills, and sore throat, but there are various other symptoms as well.",
    "Symptoms of COVID-19 include various symptoms such as fever, chills, new persistent cough, and loss or change in sense of smell or taste.",
    "Dementia has the following symptoms: experiencing memory loss, decreased judgment, and confusion.",
    "COVID-19 has the following symptoms: experiencing memory loss, decreased judgment, and confusion.",
    "COBIT 2019 is the optimal framework for establishing governance systems that support government digital government plans.",
    "Symptoms of novel coronavirus infection include various symptoms such as fever, chills, new persistent cough, and loss or change in sense of smell or taste.",
    "Symptoms of bird flu include high fever, cough, muscle pain, fatigue, headache, and other symptoms.",
    "COVID-19 is an infectious disease caused by SARS-CoV-2 (novel coronavirus). It was first identified in December 2019 and subsequently became a global pandemic. Symptoms include fever, cough, shortness of breath, fatigue, and loss of smell or taste. The infection routes are mainly droplet transmission and contact transmission, but airborne transmission is also possible. Elderly people and those with underlying medical conditions have a higher risk of severe illness and mortality. Preventive measures recommended for COVID-19 include vaccination, hand washing, wearing masks, and social distancing. The pandemic has had a significant impact on healthcare systems and economies worldwide.",
    "SGDs (Sustainable Development Goals) are 17 international goals that form the core of the '2030 Agenda for Sustainable Development' adopted by United Nations member states in 2015.",
    "A vector database is a database system specialized in representing data as vectors (arrays of numbers) and performing searches and analyses based on similarities between vectors.",
    "Notable vector databases include ChromaDB, Faiss, Pinecone, Annoy, Oracle Database 23ai AI Vector Search, and others.",
    "Some vector databases employ approximate nearest neighbor search algorithms such as HNSW for efficient searching.",
    "Approximate nearest neighbor search is an algorithm that efficiently finds the closest (most similar) data points to a given query point in high-dimensional space.",
    "Oracle Database is a relational database management system developed and sold by Oracle Corporation in the United States."
]

In [32]:
embed_text_detail.inputs = documents

### ドキュメントの埋め込み実行

In [33]:
embed_text_detail.input_type = "SEARCH_DOCUMENT" # 検索対象のドキュメント
doc_embedding = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
doc_embedding_vectors = np.array([embed for embed in doc_embedding.data.embeddings])

In [34]:
print(f"埋め込みの数:{len(doc_embedding.data.embeddings)}")
print(f"埋め込みの次元数:{len(doc_embedding.data.embeddings[0])}")

埋め込みの数:38
埋め込みの次元数:1024


In [35]:
print(f"1件目の埋め込み (最初の5要素):{doc_embedding.data.embeddings[0][:5]}")

1件目の埋め込み (最初の5要素):[-9.942055e-05, -0.00111866, 0.038757324, -0.0029335022, 0.007183075]


### クエリーの埋め込み処理

#### クエリーの定義

In [36]:
query1 = ["ベクトルデータベースとは？"]

#### クエリー1の埋め込み生成

In [37]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector1 = np.array(query_embedding1.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [38]:
scores = np.dot(query_vector1, np.transpose(doc_embedding_vectors))

#### 類似度順に並べ替えて上位5件を抽出

In [39]:
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n] # scores の要素を降順にソートしてインデックスを返す

#### 結果の表示

In [40]:
# Pandas Dataframe を使って結果を表示
df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.8246,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
1,2,0.7663,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
2,3,0.6603,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
3,4,0.5681,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです
4,5,0.5670,A vector database is a database system specialized in representing data as vectors (arrays of numbers) and performing searches and analyses based on similarities between vectors.


## 類似性検索のデモ（6） - ドキュメントの品質の考慮

### 検索対象のドキュメントの定義

In [50]:
documents = [
    "COVID-19には多くの症状があります。",
    "COVID-19の症状は悪いです。",
    "COVID-19の症状は良くありません",
    "COVID-19の症状はいろいろです",
    "COVID-19の症状は悪いです。5Gの機能には、より広範なサービスカバレッジ、より多くの利用可能な接続数、および低電力消費が含まれます。",
    "COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。",
    "COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "認知症には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COVID-19には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。",
    "新型コロナ感染症の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "鳥インフルエンザの症状には、高熱、咳、筋肉の痛み、倦怠感、頭痛、その他の症状が含まれます。",
    "COVID-19は、SARS-CoV-2（新型コロナウイルス）によって引き起こされる感染症です。2019年12月に最初に確認され、その後、世界的なパンデミックとなりました。症状としては、発熱、咳、息切れ、倦怠感、嗅覚や味覚の喪失などがあります。また、感染経路は、主に飛沫感染と接触感染ですが、空気感染の可能性もあります。高齢者や基礎疾患のある方は重症化リスクが高く、死亡率も高いとされています。COVID-19に対しては、ワクチン接種や手洗い、マスク着用、ソーシャルディスタンスなどの予防措置が推奨されています。パンデミックは世界中の医療システムや経済に大きな影響を与えました。",
    "SGDs（持続可能な開発目標）は、2015年に国連加盟国によって採択された「持続可能な開発のための2030アジェンダ」の中核を成す17の国際目標です。",
    "ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。",
    "著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。",
    "ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。",
    "近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。",
    "Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです"
]

In [51]:
embed_text_detail.inputs = documents

### ドキュメントの埋め込み実行

In [52]:
embed_text_detail.input_type = "SEARCH_DOCUMENT" # 検索対象のドキュメント
#embed_text_detail.input_type = "SEARCH_QUERY" # 検索対象のドキュメント
doc_embedding = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
doc_embedding_vectors = np.array([embed for embed in doc_embedding.data.embeddings])

In [53]:
print(f"埋め込みの数:{len(doc_embedding.data.embeddings)}")
print(f"埋め込みの次元数:{len(doc_embedding.data.embeddings[0])}")

埋め込みの数:19
埋め込みの次元数:1024


In [54]:
print(f"1件目の埋め込み (最初の5要素):{doc_embedding.data.embeddings[0][:5]}")

1件目の埋め込み (最初の5要素):[-0.0001835823, -0.0014657974, 0.03866577, -0.0029411316, 0.006828308]


### クエリーの埋め込み処理

#### クエリーの定義

In [55]:
query1 = ["COVID-19の症状は？"]

#### クエリー1の埋め込み生成

In [56]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector1 = np.array(query_embedding1.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [57]:
# 類似度スコアの計算
scores = np.dot(query_vector1, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [58]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.7968,COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。
1,2,0.7868,COVID-19には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。
2,3,0.7763,COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。
3,4,0.7398,COVID-19には多くの症状があります。
4,5,0.7379,新型コロナ感染症の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。
